In [2]:
! pip install -r requirements.txt --quiet

# Creating and Deploying an Azure AI Agent Using SDK and OpenAPI Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use Azure AI Agent Service with OpenAPI Specified Tools](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/openapi-spec?tabs=python&pivots=code-example)

In [1]:

from os import environ
from dotenv import load_dotenv
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
import requests

load_dotenv(override=True)

True

## Create a project client

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

## Create the OpenAPI Spec tool definition

Our Agent will create its tools from our deployed OpenAPI endpoints, for this to work we will need to pass in the openapi.json spec document.

In [3]:
spec_url = environ['OPENAPI_URL']
response = requests.get(spec_url)
response.raise_for_status()


openapi_spec = response.json()

auth = OpenApiAnonymousAuthDetails()

openapi = OpenApiTool(name="get_energy_usage", spec=openapi_spec, description="Retrieve data center energy, power, or consumption topic", auth=auth)


## Create an agent and a thread

In [4]:
instructions = """
You are a helpful and knowledgeable assistant that helps users understand and monitor data center energy usage.

Use the provided tools to retrieve accurate, real-time, and historical data on energy consumption, efficiency metrics (such as PUE), carbon footprint, and resource utilization across different data center regions or racks.

Always respond with concise and clear summaries, including units of measurement (e.g., kWh, %). If applicable, compare usage across regions or time ranges to highlight trends or inefficiencies.

If the user asks something unrelated to data center energy or the information is not available via the provided tools, respond with: "I'm unable to find that information based on the current data."

Only use the tools provided to retrieve answers.
"""



agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Data Center Energy Usage Agent",
    instructions=instructions,
    tools=openapi.definitions
)

print(f"Created agent, ID: {agent.id}")

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")



Created agent, ID: asst_Re5YfSoxJ8gLXywlQdfX7d15
Created thread, ID: thread_JGBmo87U0GhuewjNs5EgifVC


## Create a run and check the output

In [5]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What data centers have the highest co2 emissions?",
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Created message, ID: msg_iKtPsVo5x6RukOMwAZ7orknQ
Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_zYWPRw9HHMLFFvdj2U9bPhyo', 'object': 'thread.message', 'created_at': 1746027161, 'assistant_id': 'asst_Re5YfSoxJ8gLXywlQdfX7d15', 'thread_id': 'thread_JGBmo87U0GhuewjNs5EgifVC', 'run_id': 'run_YXlbhdZdQQHImyozDQJqTsJF', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The highest CO2 emissions among data centers are currently observed in the following cases:\n\n1. **DC-CHI2 (Zone C3)**: 153.31 kg of CO2 emitted during a period of diesel power usage, likely due to high cooling load and IT load inefficiencies.\n\n2. **DC-NYC1 (Zone B2)**: 126.48 kg of CO2 during a system powered by battery energy sources.\n\n3. **DC-CHI2 (Zone C3)**: Another spike noted at 140.93 kg of CO2, again associated with diesel energy.\n\nEfforts to mitigate these high emissions could focus on optimizing cooling systems and reducing dependency on non

## Delete the assistant when done

Comment the code below to save the agent to your Azure AI Froundry project.


In [6]:

#project_client.agents.delete_agent(agent.id)
#print("Deleted agent")